<a href="https://colab.research.google.com/github/JonathanSum/pytorch-Deep-Learning_colab/blob/master/09_echo_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<img src="https://i.imgur.com/0mFHjmZ.png" width=300>
# Signal echoing

Echoing signal `n` steps is an example of synchronized many-to-many task.

In [4]:
!git clone https://github.com/Atcold/pytorch-Deep-Learning.git 
!mv pytorch-Deep-Learning FolderPy

Cloning into 'pytorch-Deep-Learning'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 3909 (delta 13), reused 16 (delta 7), pack-reused 3875
Receiving objects: 100% (3909/3909), 93.72 MiB | 32.69 MiB/s, done.
Resolving deltas: 100% (2318/2318), done.
mv: cannot move 'pytorch-Deep-Learning' to 'FolderPy/pytorch-Deep-Learning': Directory not empty


In [0]:
from FolderPy.res.sequential_tasks import EchoData
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1);

In [0]:
batch_size = 5
echo_step = 3
series_length = 20_000
BPTT_T = 20

train_data = EchoData(
    echo_step=echo_step,
    batch_size=batch_size,
    series_length=series_length,
    truncated_length=BPTT_T,
)
train_size = len(train_data)

test_data = EchoData(
    echo_step=echo_step,
    batch_size=batch_size,
    series_length=series_length,
    truncated_length=BPTT_T,
)
test_size = len(test_data)

In [7]:
# Let's print first 20 timesteps of the first sequences to see the echo data:
print('(1st input sequence)  x:', *train_data.x_batch[0, :20], '... ')
print('(1st target sequence) y:', *train_data.y_batch[0, :20], '... ')

(1st input sequence)  x: 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 ... 
(1st target sequence) y: 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 ... 


In [8]:
# batch_size different sequences are created:
print('x_batch:', *(str(d)[1:-1] + ' ...' for d in train_data.x_batch[:, :20]), sep='\n')
print('x_batch size:', train_data.x_batch.shape)
print()
print('y_batch:', *(str(d)[1:-1] + ' ...' for d in train_data.y_batch[:, :20]), sep='\n')
print('y_batch size:', train_data.y_batch.shape)

x_batch:
0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 ...
0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 ...
1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 ...
1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 ...
1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 ...
x_batch size: (5, 20000)

y_batch:
0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 ...
0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 ...
0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 ...
0 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 ...
0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1 ...
y_batch size: (5, 20000)


In [9]:
# In order to use RNNs data is organized into temporal
# chunks of size [batch_size, T, feature_dim]
print('x_chunk:', *train_data.x_chunks[0].squeeze(), sep='\n')
print('1st x_chunk size:', train_data.x_chunks[0].shape)
print()
print('y_chunk:', *train_data.y_chunks[0].squeeze(), sep='\n')
print('1st y_chunk size:', train_data.y_chunks[0].shape)

x_chunk:
[0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0]
[0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1]
[1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1]
[1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1]
[1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1]
1st x_chunk size: (5, 20, 1)

y_chunk:
[0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0]
[0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0]
[0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0]
[0 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0]
[0 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1]
1st y_chunk size: (5, 20, 1)


In [0]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, rnn_hidden_size, output_size):
        super().__init__()
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = torch.nn.RNN(
            input_size=input_size,
            hidden_size=rnn_hidden_size,
            num_layers=1,
            nonlinearity='relu',
            batch_first=True
        )
        self.linear = torch.nn.Linear(
            in_features=rnn_hidden_size,
            out_features=1
        )

    def forward(self, x, hidden):
        x, hidden = self.rnn(x, hidden)  
        x = self.linear(x)
        return x, hidden

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def train(hidden):
    model.train()
       
    correct = 0
    for batch_idx in range(train_size):
        data, target = train_data[batch_idx]
        data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
        optimizer.zero_grad()
        if hidden is not None: hidden.detach_()
        logits, hidden = model(data, hidden)
        loss = criterion(logits, target)
        loss.backward()
        optimizer.step()
        
        pred = (torch.sigmoid(logits) > 0.5)
        correct += (pred == target.byte()).int().sum().item()
        
    return correct, loss.item(), hidden

In [0]:
def test(hidden):
    model.eval()   
    correct = 0
    with torch.no_grad():
        for batch_idx in range(test_size):
            data, target = test_data[batch_idx]
            data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).float().to(device)
            logits, hidden = model(data, hidden)
            
            pred = (torch.sigmoid(logits) > 0.5)
            correct += (pred == target.byte()).int().sum().item()

    return correct

In [0]:
feature_dim = 1 #since we have a scalar series
h_units = 4

model = SimpleRNN(
    input_size=1,
    rnn_hidden_size=h_units,
    output_size=feature_dim
).to(device)
hidden = None
        
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [15]:
n_epochs = 5
epoch = 0

while epoch < n_epochs:
    correct, loss, hidden = train(hidden)
    epoch += 1
    train_accuracy = float(correct) / train_size
    print(f'Train Epoch: {epoch}/{n_epochs}, loss: {loss:.3f}, accuracy {train_accuracy:.1f}%')

#test    
correct = test(hidden)
test_accuracy = float(correct) / test_size
print(f'Test accuracy: {test_accuracy:.1f}%')

Train Epoch: 1/5, loss: 0.670, accuracy 54.0%
Train Epoch: 2/5, loss: 0.647, accuracy 63.5%
Train Epoch: 3/5, loss: 0.502, accuracy 69.0%
Train Epoch: 4/5, loss: 0.298, accuracy 78.3%
Train Epoch: 5/5, loss: 0.395, accuracy 84.8%
Test accuracy: 85.4%


In [16]:
# Let's try some echoing
my_input = torch.empty(1, 100, 1).random_(2).to(device)
hidden = None
my_out, _ = model(my_input, hidden)
print(my_input.view(1, -1).byte(), (my_out > 0).view(1, -1), sep='\n')

tensor([[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
         0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
         0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
         1, 1, 0, 0]], dtype=torch.uint8)
tensor([[ True, False,  True, False, False, False,  True, False, False, False,
          True, False, False, False,  True,  True, False,  True,  True, False,
         False,  True, False, False,  True,  True,  True, False,  True, False,
         False,  True,  True, False,  True,  True, False,  True, False, False,
          True,  True, False,  True,  True, False,  True,  True,  True, False,
          True, False, False,  True,  True, False,  True,  True, False,  True,
         False, False,  True,  True,  True, False,  True,  True, False,  True,
          True, False,  True,  True,  True,  True, False, False,  True, False,
  